In [1]:
from datasets import load_dataset, Audio
dataset = load_dataset("synthbot/pony-speech")

Resolving data files:   0%|          | 0/31 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/31 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [2]:
from svc_helper.pitch.rmvpe import RMVPEModel
import numpy as np
import librosa

rmvpe_model = RMVPEModel(device='cuda')

def avg_pitch(entry):
    audio_entry = entry['audio']
    audio = audio_entry['array']
    resampled = librosa.resample(audio,
        orig_sr=audio_entry['sampling_rate'],
        target_sr=RMVPEModel.expected_sample_rate)
    pitch = rmvpe_model.extract_pitch(resampled)
    entry['f0_mean'] = pitch[pitch != 0].mean()
    return entry

RVC_RMVPE_PATH: D:\hf_cache\hub\models--therealvul--svc_helper\snapshots\b1a77cd33e055a661ee1c81ffff6a59178a7419b\rvc_rmvpe.pt


In [10]:
#f0_means = dataset['train'].select(range(10)).map(avg_pitch)
f0_means = dataset['train'].filter(lambda ex: ex['speaker'] == 'Rainbow Dash',
    num_proc=12).map(avg_pitch)
import numpy as np
np.nanmean(np.array(f0_means['f0_mean']))

Filter (num_proc=12):   0%|          | 0/64659 [00:00<?, ? examples/s]

Map:   0%|          | 0/5533 [00:00<?, ? examples/s]

C:\Users\vul\AppData\Local\Temp\ipykernel_29408\2948678516.py:14: RuntimeWarning: Mean of empty slice.
  entry['f0_mean'] = pitch[pitch != 0].mean()
c:\Users\vul\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


412.24502967803943